In [1]:
import ibis
from google.oauth2 import service_account
from google.cloud import bigquery
from pathlib import Path

credentials = service_account.Credentials.from_service_account_file( "sol-eng-nickp-a7734a7e7018.json" )

client = bigquery.Client(credentials=credentials)

ibiscon = ibis.bigquery.connect(
    project_id="sol-eng-nickp",
    dataset_id="loan_data",
    credentials=credentials
)


In [2]:
feature_columns = ["INT_RATE", "TERM", "BC_UTIL", "BC_OPEN_TO_BUY", "ALL_UTIL", "ID"]

lendingclub_data = ibiscon.table("loan_data")
lendingclub_data = lendingclub_data.drop_null(feature_columns, how="any")

# Grab a sample of the lending club data and download it to a local Pandas DataFrame
lendingclub_sample = lendingclub_data[feature_columns]\
    .sample(.003)\
    .to_pandas()

lendingclub_sample.head(5)

,INT_RATE,TERM,BC_UTIL,BC_OPEN_TO_BUY,ALL_UTIL,ID
0,13.56,36 months,64.0,12252.0,65.0,141625106
1,7.46,36 months,27.2,70273.0,23.0,135118830
2,11.80,60 months,21.4,98610.0,32.0,144754520
3,7.56,36 months,18.6,54349.0,18.0,144413158
4,10.72,36 months,3.7,68374.0,18.0,143494798


In [3]:
#  Split our sample into training and test datasets.
from sklearn.model_selection import train_test_split

target_column = "INT_RATE"

X_train, X_test, y_train, y_test = train_test_split(
    lendingclub_sample.drop(target_column, axis=1), lendingclub_sample[target_column], test_size=0.2, random_state=42
)

In [4]:
# Create a feature engineering and model pipeline
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression

noncategorical_columns = ["BC_UTIL", "BC_OPEN_TO_BUY", "ALL_UTIL"]

pipeline = Pipeline(
    [
        (
            "preprocess", 
            ColumnTransformer(
                [
                    # One-hot encode "TERM", our only categorical variables
                    ("encoder", OrdinalEncoder(), ["TERM"]),
                    # Mean imputation on all missing data
                    ("mean_imputation", SimpleImputer(missing_values=np.nan, strategy='mean'), noncategorical_columns),
                    # Scale all columns
                    ("scaler", StandardScaler(with_std=False), noncategorical_columns),
                ],
                remainder="passthrough")
        ),
        (
            "linear_regression", 
            LinearRegression()
        ),
    ]
)
pipeline.fit(X_train, y_train)

/home/nick.pelikan/posit-gcp-python/.venv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder', OrdinalEncoder(),
                                                  ['TERM']),
                                                 ('mean_imputation',
                                                  SimpleImputer(),
                                                  ['BC_UTIL', 'BC_OPEN_TO_BUY',
                                                   'ALL_UTIL']),
                                                 ('scaler',
                                                  StandardScaler(with_std=False),
                                                  ['BC_UTIL', 'BC_OPEN_TO_BUY',
                                                   'ALL_UTIL'])])),
                ('linear_regression', LinearRegression())])

In [5]:
# Convert the fit pipeline into an orbital object
import orbitalml
import orbitalml.types

orbitalml_pipeline = orbitalml.parse_pipeline(pipeline, features={
    "ID": orbitalml.types.DoubleColumnType(),
    "TERM": orbitalml.types.StringColumnType(),
    "BC_UTIL": orbitalml.types.DoubleColumnType(),
    "BC_OPEN_TO_BUY": orbitalml.types.DoubleColumnType(),
    "ALL_UTIL": orbitalml.types.DoubleColumnType(),
})

In [6]:
from ibis.expr.sql import SQLString
sql = orbitalml.export_sql("TEST_TABLE", orbitalml_pipeline, dialect="bigquery")

SQLString(sql)

/home/nick.pelikan/posit-gcp-python/.venv/lib/python3.12/site-packages/orbitalml/translation/steps/labelencoder.py:66: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  case_expr = ibis.case()


```sql
SELECT `t0`.`ID` AS `ID`, CAST(CAST(CASE WHEN `t0`.`TERM` = '36 months' THEN 0 WHEN `t0`.`TERM` = '60 months' THEN 1 ELSE -1 END AS FLOAT64) AS FLOAT64) * 3.786192938761481 + 10.566732320209402 + COALESCE(`t0`.`BC_UTIL`, 54.615333557128906) * 0.007302144293509861 + COALESCE(`t0`.`BC_OPEN_TO_BUY`, 12757.642578125) * -3.0921038588161445e-05 + COALESCE(`t0`.`ALL_UTIL`, 57.12452697753906) * 0.012627186393865421 + (`t0`.`BC_UTIL` - 54.61533313936573) * 0.007302146307449486 + (`t0`.`BC_OPEN_TO_BUY` - 12757.643002618563) * -3.092104273484158e-05 + (`t0`.`ALL_UTIL` - 57.12452720395694) * 0.012627186393862795 + `t0`.`ALL_UTIL` * 4.844310072006641e-09 AS `variable` FROM `TEST_TABLE` AS `t0`
```

In [7]:
import pandas_gbq

result = pandas_gbq.read_gbq(
    sql.replace("`TEST_TABLE`", "loan_data.loan_data"),
    project_id="sol-eng-nickp",
    progress_bar_type='tqdm_notebook',
    credentials=credentials
)
result.dropna().head(20)

Downloading:   0%|          |

,ID,variable
0,96565245,7.262981
1,96556185,9.430262
2,96513653,10.699812
3,96133729,16.527024
4,96558415,10.869084
5,96204284,12.168855
6,96281265,12.502974
7,96466415,10.369929
8,96013928,12.994114
9,96241251,16.701975


In [13]:
from google.cloud.bigquery import dbapi

con = dbapi.Connection(client=client)

cursor = con.cursor()
cursor.execute(
    f"""
    CREATE OR REPLACE VIEW sol-eng-nickp.loan_data.pyorbital_test_v0 AS {sql.replace("`TEST_TABLE`", "sol-eng-nickp.loan_data.loan_data")};
    """
)
# 